all exercises based on [Deep Learning for NLP](https://wikidocs.net/45101)

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

## 1) 데이터 이해와 전처리

In [42]:
text = """경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다"""

In [43]:
t = Tokenizer()
t.fit_on_texts([text]) # 리스트로 넣어야 함
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기: %d' % vocab_size)

단어 집합의 크기: 12


In [44]:
print(t.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [45]:
t.texts_to_sequences(['말이 경마장에 가는 법이다람쥐'])

[[1, 2, 8]]

In [46]:
sequences = list()
for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        
print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 11


In [47]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [48]:
max_len = max(len(l) for l in sequences)

print('샘플의 최대 길이: {}'.format(max_len))
# 이 최대 길이로 샘플 길이 padding

샘플의 최대 길이: 6


In [49]:
sequences = pad_sequences(sequences, maxlen = max_len, padding = 'pre')

In [50]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [51]:
# 마지막 label 데이터 분리
sequences = np.array(sequences)
X = sequences[:, :-1]
y = sequences[:, -1]
# 리스트의 마지막 값을 제외하고 저장한 건 X
# 리스트의 마지막 값만 저장한 건 y (레이블)

In [52]:
print(X)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


In [53]:
print(y)

[ 3  1  4  5  1  7  1  9 10  1 11]


In [54]:
# 레이블을 원핫인코딩 해줘야해
y = to_categorical(y, num_classes=vocab_size)

In [55]:
print(y) # vocab_size = 12 

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


## 2) 모델 설계하기

In [56]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [57]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length = max_len-1))
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X, y, epochs = 200, verbose = 2)

Epoch 1/200
1/1 - 0s - loss: 2.5425 - accuracy: 0.1818
Epoch 2/200
1/1 - 0s - loss: 2.5245 - accuracy: 0.1818
Epoch 3/200
1/1 - 0s - loss: 2.5076 - accuracy: 0.1818
Epoch 4/200
1/1 - 0s - loss: 2.4918 - accuracy: 0.2727
Epoch 5/200
1/1 - 0s - loss: 2.4769 - accuracy: 0.2727
Epoch 6/200
1/1 - 0s - loss: 2.4627 - accuracy: 0.1818
Epoch 7/200
1/1 - 0s - loss: 2.4491 - accuracy: 0.1818
Epoch 8/200
1/1 - 0s - loss: 2.4358 - accuracy: 0.1818
Epoch 9/200
1/1 - 0s - loss: 2.4229 - accuracy: 0.2727
Epoch 10/200
1/1 - 0s - loss: 2.4101 - accuracy: 0.3636
Epoch 11/200
1/1 - 0s - loss: 2.3973 - accuracy: 0.5455
Epoch 12/200
1/1 - 0s - loss: 2.3845 - accuracy: 0.6364
Epoch 13/200
1/1 - 0s - loss: 2.3715 - accuracy: 0.6364
Epoch 14/200
1/1 - 0s - loss: 2.3583 - accuracy: 0.6364
Epoch 15/200
1/1 - 0s - loss: 2.3448 - accuracy: 0.6364
Epoch 16/200
1/1 - 0s - loss: 2.3309 - accuracy: 0.6364
Epoch 17/200
1/1 - 0s - loss: 2.3165 - accuracy: 0.6364
Epoch 18/200
1/1 - 0s - loss: 2.3016 - accuracy: 0.6364
E

Epoch 147/200
1/1 - 0s - loss: 0.2210 - accuracy: 1.0000
Epoch 148/200
1/1 - 0s - loss: 0.2163 - accuracy: 1.0000
Epoch 149/200
1/1 - 0s - loss: 0.2116 - accuracy: 1.0000
Epoch 150/200
1/1 - 0s - loss: 0.2071 - accuracy: 1.0000
Epoch 151/200
1/1 - 0s - loss: 0.2027 - accuracy: 1.0000
Epoch 152/200
1/1 - 0s - loss: 0.1984 - accuracy: 1.0000
Epoch 153/200
1/1 - 0s - loss: 0.1942 - accuracy: 1.0000
Epoch 154/200
1/1 - 0s - loss: 0.1901 - accuracy: 1.0000
Epoch 155/200
1/1 - 0s - loss: 0.1861 - accuracy: 1.0000
Epoch 156/200
1/1 - 0s - loss: 0.1822 - accuracy: 1.0000
Epoch 157/200
1/1 - 0s - loss: 0.1784 - accuracy: 1.0000
Epoch 158/200
1/1 - 0s - loss: 0.1747 - accuracy: 1.0000
Epoch 159/200
1/1 - 0s - loss: 0.1711 - accuracy: 1.0000
Epoch 160/200
1/1 - 0s - loss: 0.1675 - accuracy: 1.0000
Epoch 161/200
1/1 - 0s - loss: 0.1641 - accuracy: 1.0000
Epoch 162/200
1/1 - 0s - loss: 0.1607 - accuracy: 1.0000
Epoch 163/200
1/1 - 0s - loss: 0.1574 - accuracy: 1.0000
Epoch 164/200
1/1 - 0s - loss: 

In [58]:
def sentence_generation(model, t, current_word, n):
    # 모델, 토크나이저, 현재 단어, 반복 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기 위해 저장
    sentence = ''
    for _ in range(n):
        encoded = t.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen = 5, padding = 'pre')
        result = model.predict_classes(encoded, verbose=0) # 이 모델에는 입력값 크기가 padding된 그 크기여야 하는군
        for word, index in t.word_index.items():
            if index == result:
                break
        current_word = current_word + ' ' + word # 현재 단어 + ' ' + 예측 단어 를 current word로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
        
    sentence = init_word + sentence
    return sentence

In [59]:
print(sentence_generation(model, t, '경마장에', 4))
# 경마장에 라는 단어 뒤에 총 4개의 단어가 있으니 4번 예측해보자

경마장에 있는 말이 뛰고 있다


In [60]:
print(sentence_generation(model, t, '그의', 2))

그의 말이 법이다


In [61]:
print(sentence_generation(model, t, '가는', 5))

가는 말이 고와야 오는 말이 곱다


### LSTM으로 text 생성
더 많은 데이터로 텍스트 생성할 것!

### 1) 데이터 이해와 전처리

In [62]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [63]:
df = pd.read_csv('ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [64]:
print('열의 개수:', len(df.columns))
print(df.columns)

열의 개수: 15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [65]:
# headline 열 활용할 것.
# null 값 확인
df['headline'].isnull().sum()
# .isnull().values.any()

0

In [66]:
headline = [] # 여기에 모든 헤드라인 모을 것
headline.extend(list(df.headline.values)) # 헤드라인의 값들을 headline에 리스트로 저장
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

'Unknow' 발견

지금 진행하는 내용에서 null값이나 다름 없음 -> 없애자


In [67]:
headline.count('Unknown')

110

In [68]:
print('제거 전 총 샘플 개수:', len(headline))

제거 전 총 샘플 개수: 1324


In [69]:
headline = [n for n in headline if n != 'Unknown']
print('제거 후 총 샘플 개수:', len(headline))

제거 후 총 샘플 개수: 1214


In [70]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [71]:
# 이제 데이터 전처리.
# 구두점 제거 + 단어 소문자화

def repreprocessing(s):
    s = s.encode('utf8').decode('ascii', 'ignore')
    return ''.join(c for c in s if c not in punctuation).lower()

text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [72]:
# 단어집합 만들고 크기 보자
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index)+1
print('단어 집합의 크기: ',vocab_size)

단어 집합의 크기:  3494


In [73]:
sequences = list()
for line in text: # text 1214개
    encoded = t.texts_to_sequences([line])[0] # 결과가 이중리스트라서 [0]은 별 의미 안둬도 되나?
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        
sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [74]:
# 인덱스로부터 단어를 찾는 딕셔너리 만들기
index_to_word = {}
for key, value in t.word_index.items():
    index_to_word[value] = key
    
print('빈도수 상위 582번 단어: {}'.format(index_to_word[582]))

빈도수 상위 582번 단어: offer


In [75]:
# 패딩 작업
max_len = max(len(l) for l in sequences)
print('샘플 최대 길이:', max_len) # 24로 패딩할 것이다

샘플 최대 길이: 24


In [76]:
sequences = pad_sequences(sequences, maxlen = max_len, padding = 'pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [77]:
#이제 레이블 분리
sequences = np.array(sequences)
X = sequences[:, :-1]
y = sequences[:, -1]
print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


In [78]:
y = to_categorical(y, num_classes = vocab_size)

### 2) 모델 설계하기

In [79]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [80]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length = max_len -1))
model.add(LSTM(128))
model.add(Dense(vocab_size, activation = 'softmax'))
model.compile(loss= 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs = 200, verbose = 2)

Epoch 1/200
244/244 - 3s - loss: 7.6447 - accuracy: 0.0261
Epoch 2/200
244/244 - 3s - loss: 7.1135 - accuracy: 0.0309
Epoch 3/200
244/244 - 3s - loss: 6.9718 - accuracy: 0.0352
Epoch 4/200
244/244 - 3s - loss: 6.8396 - accuracy: 0.0383
Epoch 5/200
244/244 - 3s - loss: 6.6829 - accuracy: 0.0424
Epoch 6/200
244/244 - 3s - loss: 6.4993 - accuracy: 0.0461
Epoch 7/200
244/244 - 3s - loss: 6.2966 - accuracy: 0.0531
Epoch 8/200
244/244 - 3s - loss: 6.0852 - accuracy: 0.0590
Epoch 9/200
244/244 - 3s - loss: 5.8854 - accuracy: 0.0611
Epoch 10/200
244/244 - 3s - loss: 5.6942 - accuracy: 0.0672
Epoch 11/200
244/244 - 3s - loss: 5.5098 - accuracy: 0.0751
Epoch 12/200
244/244 - 3s - loss: 5.3409 - accuracy: 0.0796
Epoch 13/200
244/244 - 3s - loss: 5.1828 - accuracy: 0.0824
Epoch 14/200
244/244 - 3s - loss: 5.0322 - accuracy: 0.0936
Epoch 15/200
244/244 - 3s - loss: 4.8875 - accuracy: 0.1038
Epoch 16/200
244/244 - 4s - loss: 4.7473 - accuracy: 0.1143
Epoch 17/200
244/244 - 4s - loss: 4.6164 - accura

244/244 - 4s - loss: 0.3402 - accuracy: 0.9163
Epoch 138/200
244/244 - 4s - loss: 0.3372 - accuracy: 0.9150
Epoch 139/200
244/244 - 4s - loss: 0.3343 - accuracy: 0.9162
Epoch 140/200
244/244 - 5s - loss: 0.3304 - accuracy: 0.9166
Epoch 141/200
244/244 - 6s - loss: 0.3286 - accuracy: 0.9164
Epoch 142/200
244/244 - 6s - loss: 0.3252 - accuracy: 0.9161
Epoch 143/200
244/244 - 5s - loss: 0.3237 - accuracy: 0.9163
Epoch 144/200
244/244 - 6s - loss: 0.3227 - accuracy: 0.9166
Epoch 145/200
244/244 - 5s - loss: 0.3184 - accuracy: 0.9159
Epoch 146/200
244/244 - 5s - loss: 0.3161 - accuracy: 0.9157
Epoch 147/200
244/244 - 4s - loss: 0.3137 - accuracy: 0.9164
Epoch 148/200
244/244 - 4s - loss: 0.3127 - accuracy: 0.9170
Epoch 149/200
244/244 - 4s - loss: 0.3125 - accuracy: 0.9140
Epoch 150/200
244/244 - 4s - loss: 0.3084 - accuracy: 0.9159
Epoch 151/200
244/244 - 4s - loss: 0.3071 - accuracy: 0.9144
Epoch 152/200
244/244 - 4s - loss: 0.3036 - accuracy: 0.9167
Epoch 153/200
244/244 - 4s - loss: 0.3

In [81]:
def sentence_generation(model, t, current_word, n):
    # 모델, 토크나이저, 현재 단어, 반복 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기 위해 저장
    sentence = ''
    for _ in range(n):
        encoded = t.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen = 23, padding = 'pre')
        result = model.predict_classes(encoded, verbose=0) # 이 모델에는 입력값 크기가 padding된 그 크기여야 하는군
        for word, index in t.word_index.items():
            if index == result:
                break
        current_word = current_word + ' ' + word # 현재 단어 + ' ' + 예측 단어 를 current word로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
        
    sentence = init_word + sentence
    return sentence

In [82]:
print(sentence_generation(model, t, 'i', 10))

i cant jump ship from facebook yet snarl and a meeting


In [83]:
print(sentence_generation(model, t, 'how', 10))

how to make facebook more accountable over epa at ryan attack
